In [5]:
from bs4 import BeautifulSoup
import pandas as pd
import random
import time
import math
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta

In [4]:
# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")

In [ ]:
# 用來把資料存在各個list裡的function，之後會建成dataframe
def best_seller_function(gridItemRoot, department_list, department_str, rank, title, book_url, author, price, time_scrape):
    # department
    department_list.append(department_str)

    # rank
    rank.append(gridItemRoot.find("span", class_="zg-bdg-text").text)

    # title
    try:
        title.append(gridItemRoot.find("div", class_="_cDEzb_p13n-sc-css-line-clamp-2_EWgCb").text)
    except AttributeError:
        try:
            title.append(gridItemRoot.find("div", class_="_cDEzb_p13n-sc-css-line-clamp-3_g3dy1").text)
        except AttributeError:
            try:
                title.append(gridItemRoot.find("div", class_="_cDEzb_p13n-sc-css-line-clamp-4_2q2cc").text)
            except AttributeError:
                try:
                    title.append(gridItemRoot.find_all("div", class_="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")[0].text)
                except IndexError:
                    title.append(gridItemRoot.find("div", class_="_cDEzb_p13n-sc-css-line-clamp-5_2l-dX").text)

    # book url
    book_url.append(f"https://www.amazon.com{gridItemRoot.find_all('a',class_='a-link-normal')[1].get('href')}")

    # author
    try:
        author.append(gridItemRoot.find_all("div", class_="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")[1].text)
    except IndexError:
        try:
            author.append(gridItemRoot.find("div", class_="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").text)
        # 如果沒有author，就以"_"標記
        except AttributeError:
            author.append("_")

    # format
    try:
        format.append(gridItemRoot.find("span", class_="a-size-small a-color-secondary a-text-normal").text)
    # 如果沒有format，就以"_"標記
    except AttributeError:
        format.append("_")

    # price
    try:
        price.append(gridItemRoot.find("span", class_="_cDEzb_p13n-sc-price_3mJ9Z").text)
    except AttributeError:
        try:
            a_color_secondary = gridItemRoot.find_all("span", class_="a-color-secondary")[-1].text

            # 由於a-color-secondary可能會爬到別的東西，辨別價格方法是有沒有$
            if a_color_secondary.count("$") == 1:
                price.append(a_color_secondary)
            else:
                price.append("_")
        except IndexError:
            price.append("_")

    # scrape time
    time_scrape.append(datetime.now() + timedelta(hours=8))

In [ ]:
#Input

url = []
# 讀取要爬的類別總榜前100
with open('url.txt','r') as f:
    for row in f:
      if row != '\n':
        url.append(row.rstrip('\n'))

In [ ]:
# 建空的list
# url_conti: 存下一層類別的網址
# departments: 存所屬類別
# rank: 存類別排名
# title: 存書名
# book_url: 存商品頁面網址
# format: 存書的樣式(如paperback、hardcover、kindle等)
# price: 存價格
# time_scrape: 存爬取時間
url_conti, departments, rank, title, book_url, author, format, price, time_scrape =[],[],[],[],[],[],[],[],[]

In [ ]:
# 開啟selenium chrome driver
##############記得要把語言改成英文，不然價格就不是美金
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
                          options=chrome_options)
driver.get('https://www.amazon.com/ref=nav_logo')

In [ ]:
# 斷掉的話要改index
for row in url[:]:
    # 將欲爬網站輸入selenium chrome driver
    driver.get(row)
    # 爬Amazon慣例會先重新載入一次，大幅減少錯誤頁面機率
    driver.refresh()
    i = 0
    while True:
        # 要selemium向下滾動的code
        time.sleep(1)
        driver.execute_script("window.scrollBy(0, 5000);")
        time.sleep(1)
        driver.execute_script("window.scrollBy(0, 3000);")
        time.sleep(2)
        driver.execute_script("window.scrollBy(0, 2500);")
        time.sleep(6)
        driver.execute_script("window.scrollBy(0, 3000);")
        time.sleep(2)
        driver.execute_script("window.scrollBy(0, 2560);")
        time.sleep(4)
        # 將現在網頁內容裝進python的soup內
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        i = i+1
        print(i, end = ' ')
        # 錯誤頁面
        refuse_request = ['Request was throttled. Please wait a moment and refresh the page',
                          '      503 - Service Unavailable Error']
        if soup.text.replace('\n','').count("Sorry, we just need to make sure you're not a robot.") == 1:
          driver.refresh()
          time.sleep(30*i)

        if soup.text.replace('\n','').count("500 - An error occurred") == 1:
          driver.refresh()
          time.sleep(30*i)
        
        if soup.text.replace('\n','') in refuse_request:
          driver.refresh()
          time.sleep(30*i)

        # 如果沒有錯誤頁面，就只有1次的while迴圈
        else:
          break
    
    #找出左側的類別欄
    div = soup.find_all('div', class_='_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz')
    group = div[-1].find_all('div', class_='_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-large__1z5B8')
    non_sub_department_str = '<span class="_p13n-zg-nav-tree-all_style_zg-selected__1SfhQ">'

    # 爬類別名稱
    high_department = ''

    # 爬較高順位之類別名稱
    try:
      _p13n = div[0].find_all('div',class_='_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-large__1z5B8 _p13n-zg-nav-tree-all_style_zg-browse-up__XTlqh')
      for p13n in _p13n:
        # 以|隔開
        high_department = high_department + p13n.find('a').text + ' | '

    except AttributeError:
      _p13n = div[0].find('div',class_='_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-large__1z5B8 _p13n-zg-nav-tree-all_style_zg-browse-up__XTlqh')
      high_department = high_department + _p13n.find('a').text + ' | '

    # 爬較低順位之類別
    high_department = high_department + div[-2].find('div',class_='_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-large__1z5B8').text
    try:
      department = div[-1].find('span',class_='_p13n-zg-nav-tree-all_style_zg-selected__1SfhQ').text
      high_department = high_department + ' | '+ department
      high_department = high_department.replace('Books | ','',1)
    except AttributeError:
      high_department = high_department.replace('Books | ','',1)

    # 爬想要的資訊
    gridRow = soup.find_all('div',class_='a-column a-span12 a-text-center _cDEzb_grid-column_2hIsc')
    for gridItemRoot in gridRow:
      if gridItemRoot.text == 'This item is no longer available':
            continue
      best_seller_function(gridItemRoot, departments, high_department, rank, title, book_url, author, price, time_scrape)

    #爬第二頁(50-100名)
    try:
      li = soup.find('li',class_='a-normal').find('a').get('href')
      url_end_2 = f"https://www.amazon.com{li}"
      driver.get(url_end_2)
      driver.refresh()
      i = 0
      while True:

            time.sleep(1)
            driver.execute_script("window.scrollBy(0, 5000);")
            time.sleep(1)
            driver.execute_script("window.scrollBy(0, 3000);")
            time.sleep(2)
            driver.execute_script("window.scrollBy(0, 2500);")
            time.sleep(6)
            driver.execute_script("window.scrollBy(0, 3000);")
            time.sleep(2)
            driver.execute_script("window.scrollBy(0, 2560);")
            time.sleep(4)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            i = i+1
            print(i, end = ' ')
            refuse_request = ['Request was throttled. Please wait a moment and refresh the page',
                              '      503 - Service Unavailable Error']
            if soup.text.replace('\n','').count("Sorry, we just need to make sure you're not a robot.") == 1:
              driver.refresh()
              time.sleep(30*i)
            
            if soup.text.replace('\n','') in refuse_request:
              driver.refresh()
              time.sleep(30*i)

            else:
              break
      gridRow = soup.find_all('div',class_='a-column a-span12 a-text-center _cDEzb_grid-column_2hIsc')
      for gridItemRoot in gridRow:
        if gridItemRoot.text == 'This item is no longer available':
              continue
        best_seller_function(gridItemRoot, departments, high_department, rank, title, book_url, author, price, time_scrape)
    except AttributeError:
      url_end_2 = '_'

    # 如果有爬到，就會印出這些字串
    print('---[',url.index(row),']', high_department, 'appends best sellers url')

    # 爬下一層類別的網址
    for treeitem in group:
      try:
        url_conti.append(f'http://www.amazon.com{treeitem.find("a").get("href")}')
      except AttributeError:
        continue
    print(url.index(row),'---new')

# 爬完關掉selenium chrome
driver.quit()

In [ ]:
# 檢查資料長度一樣
print(len(rank), len(title), len(book_url), len(author), len(format), len(price), len(time_scrape))

In [ ]:
# Output

# 裝成df並儲存
Best_Sellers_df = pd.DataFrame({
    'department':departments,
    'rank': rank,
    'title':title,
    'book_url':book_url,
    'author':author,
    'format':format,
    'price':price,
    'scrape_time':time_scrape})

# 檢查是否爬重複
Best_Sellers_df = Best_Sellers_df.drop_duplicates(subset=['department','title','book_url','author','format'])






# 存檔
Best_Sellers_df.to_csv('Best_Sellers_1dep.csv',index=False)

# 存下一層類別排名網址
with open("url_2.txt",'w') as f:
    for row in url_conti:
        f.write("%s\n" % row)

In [ ]:
# 檢查是否漏抓第50名和第100名資料
Best_Sellers_df.drop_duplicates(subset = ['department'])['department']
df_rank50 = list(Best_Sellers_df[Best_Sellers_df['rank'] == '#50'].drop_duplicates(subset = ['department'])['department'])
df_rank100 = list(Best_Sellers_df[Best_Sellers_df['rank'] == '#100'].drop_duplicates(subset = ['department'])['department'])

find_not_append_dep = []
for x in list(Best_Sellers_df.drop_duplicates(subset = ['department'])['department']):
    if x not in df_rank50:
        find_not_append_dep.append(x)
    if x not in df_rank100:
        find_not_append_dep.append(x)

find_not_append_dep = [x.split(' | ')[-1].replace(' ','-') for x in find_not_append_dep]
find_not_append_dep